In [1]:
from datetime import datetime, date
import pandas as pd
from pyspark.sql import Row


In [2]:
spark

In [3]:
f = open('README.md', 'r')
data = f.read()
f.close()

In [4]:
# 文字列のカウント
data.count("\n")
moziretu = ('hogehoge1')
moziretu.count('1'),moziretu.count('hoge'),moziretu.count('g1'),

(1, 2, 0)

In [5]:
f = open('README.md', 'r')

datalist = f.readlines()
print (datalist[0])
print (datalist[1])
print (datalist[2])

f.close()
print(datalist[0])

# Apache Spark



Spark is a unified analytics engine for large-scale data processing. It provides

# Apache Spark



In [6]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [7]:
from datetime import datetime, date
import pandas as pd
from pyspark.sql import Row

df = spark.createDataFrame([
    Row(a=1, b=2., c='string1', d=date(2000, 1, 1), e=datetime(2000, 1, 1, 12, 0)),
    Row(a=2, b=3., c='string2', d=date(2000, 2, 1), e=datetime(2000, 1, 2, 12, 0)),
    Row(a=4, b=5., c='string3', d=date(2000, 3, 1), e=datetime(2000, 1, 3, 12, 0))
])
df

DataFrame[a: bigint, b: double, c: string, d: date, e: timestamp]

In [8]:
print(df)
# All DataFrames above result same.
df.show()
# df.printSchema()

DataFrame[a: bigint, b: double, c: string, d: date, e: timestamp]
+---+---+-------+----------+-------------------+
|  a|  b|      c|         d|                  e|
+---+---+-------+----------+-------------------+
|  1|2.0|string1|2000-01-01|2000-01-01 12:00:00|
|  2|3.0|string2|2000-02-01|2000-01-02 12:00:00|
|  4|5.0|string3|2000-03-01|2000-01-03 12:00:00|
+---+---+-------+----------+-------------------+



In [9]:
df.show(1)

+---+---+-------+----------+-------------------+
|  a|  b|      c|         d|                  e|
+---+---+-------+----------+-------------------+
|  1|2.0|string1|2000-01-01|2000-01-01 12:00:00|
+---+---+-------+----------+-------------------+
only showing top 1 row



In [11]:

f = open('README.md', 'r')
data = f.read()
f.close()
lines = sc.textFile('README.md')

In [12]:
# pyspark.RDD.map
rdd = sc.parallelize(['red','blue', 'green'], 3)
sorted(rdd.map(lambda x: (x, 1)).collect())

[('blue', 1), ('green', 1), ('red', 1)]

In [13]:
# pyspark.RDD.sample
rdd = sc.parallelize(range(100), 4)
6 <= rdd.sample(False, fraction=0.1, seed=81).count() <= 14

True

In [ ]:
# print(data.replace('\n', ''))
tmp = data.replace('\n', '').split(' ')
tmpdict={}
tmp
for ele in tmp:
    if ele not in tmpdict.keys():
        tmpdict[ele] = 1
    else :
        tmpdict[ele] += 1
tmpdict


In [15]:
values_sorted = sorted(dict.values())
print(values_sorted)

TypeError: unbound method dict.values() needs an argument

##### ※細かい処理の内容が気になる方は詳解Apach sparkを購入してください。

### 変換
##### RDDに処理を加えて新しいRDDを作成する

In [1]:
# pyspark.SparkContext.parallelize
# コレクションからのパーティション数5でRDDの作成
nums = sc.parallelize([0, 2, 3, 4, 6], 5).glom().collect()

In [2]:
nums

[[0], [2], [3], [4], [6]]

In [22]:
# zip(RDD,RDD) -> RDD
array1 = sc.parallelize([0, 2, 4, 6, 8], 5)
array2 = sc.parallelize([1, 3, 5, 7, 9], 5)

# RDDs have the same number of partitions and the same number of elements in each partition
array1.zip(array2)
array1.zip(array2).collect()

[(0, 1), (2, 3), (4, 5), (6, 7), (8, 9)]

In [3]:
# 和集合
# パーティションの結合は行われない。パーティション間でデータをやり取りする必要があるかが判断基準になりそう
rdd = sc.parallelize([1, 1, 2, 3])
rdd.union(rdd).collect()

[1, 1, 2, 3, 1, 1, 2, 3]

In [ ]:
# RDD.subtract(other, numPartitions=None)
# otherに含まれないselfを返す
x = sc.parallelize([("a", 1), ("b", 4), ("b", 5), ("a", 3)])
y = sc.parallelize([("a", 3), ("c", None)])
sorted(x.subtract(y).collect())

In [5]:
# pyspark.RDD.cartesian (直積)
# RDD.cartesian(other)
rdd = sc.parallelize([1, 2])
print(sorted(rdd.cartesian(rdd).collect()))
rdd = sc.parallelize([1, 2, 3])
print(sorted(rdd.cartesian(rdd).collect()))


[(1, 1), (1, 2), (2, 1), (2, 2)]
[(1, 1), (1, 2), (1, 3), (2, 1), (2, 2), (2, 3), (3, 1), (3, 2), (3, 3)]


### アクション
##### **ワーカでの処理結果をまとめ上げ**て、通常のコレクションへの変換、外部ストレージへの値の保存をする。

In [7]:
# pyspark.RDD.collect
# パーティションのRDDをドライバプログラムにすべて返却してしまう
rdd = sc.parallelize([1, 1, 2, 3])
rdd.union(rdd).collect()

[1, 1, 2, 3, 1, 1, 2, 3]

In [11]:
# count
# RDDの要素数を返却
rdd = sc.parallelize([1, 1, 2, 3])
rdd.count()

4

### アクション 集約

In [20]:
# RDDの要素を集約。各パーティションで集約を行い、全パーティションで集約の結果をマージ
from operator import add
sc.parallelize([1, 2, 3, 4, 5]).reduce(add)
sc.parallelize((2 for _ in range(10))).map(lambda x: 5).cache().reduce(add)
sc.parallelize([]).reduce(add)

ValueError: Can not reduce() empty RDD

In [28]:
# RDD.fold(zeroValue, op)
from operator import add
print(sc.parallelize([1, 2, 3, 4, 5]).fold(0, add))
print(sc.parallelize([],).fold(100, add))

15
2100


In [36]:
# パーティション内での集約時にzerovalueの値が加算される
# スライスする線+1=パーティション数
print(sc.parallelize([],numSlices=15).fold(100, add))
print(sc.parallelize([],numSlices=5).fold(100, add))

1600
600


In [40]:
# sum
sc.parallelize([1.0, 2.0, 3.0]).sum()
# stats()
# -> pyspark.statcounter.StatCounter
stats = sc.parallelize([1.0, 2.0, 3.0]).stats()

In [62]:
stats

(count: 3, mean: 2.0, stdev: 0.816496580927726, max: 3.0, min: 1.0)

### ペアRDD
- 各要素がキーとバリューのペアからなるタプルで構成されるRDD
- キーの集約時にキーのハッシュ地に基づいて同一パーティションにデータを集めることができる
- ユースケース：日付ごとの売り上げ等

In [ ]:
map キーの変更なしでもパーティションを失う

In [7]:
# pyspark.RDD.mapでペアRDDを作成
rdd = sc.parallelize(['a','b','c','d','e'])
pairrdd = rdd.map(lambda x: (x, 1))
pairrdd.collect()

[('a', 1), ('b', 1), ('c', 1), ('d', 1), ('e', 1)]

In [ ]:
mapvalues パーティションを保つ

In [1]:
x = sc.parallelize([("a", ["apple", "banana", "lemon"]), ("b", ["grapes"])])
def f(x): return len(x)
x.mapValues(f).collect()


[('a', 3), ('b', 1)]

In [ ]:
RDD.mapPartitions(f, preservesPartitioning=False)

In [4]:
rdd = sc.parallelize([1, 2, 3, 4, 5, 6, 7], 2)
def f(iterator): yield sum(iterator)
rdd.mapPartitions(f,preservesPartitioning=True).collect()

[6, 22]

In [ ]:
reduceは値を返す

In [15]:
from operator import add
type(sc.parallelize([1, 2, 3, 4, 5]).reduce(add))

int

In [ ]:
reducebykeyはRDDを返す

In [1]:
from operator import add
rdd = sc.parallelize([("a", 1), ("b", 1), ("a", 1)])
type(rdd.reduceByKey(add))

pyspark.rdd.PipelinedRDD

In [14]:
sorted(rdd.reduceByKey(add).collect())

[('a', 2), ('b', 1)]

In [ ]:
foldは値を返す foldbykeyはRDDを返す

In [6]:
print(sc.parallelize([1, 2, 3, 4, 5]).fold(0, add))

15


In [39]:
rdd = sc.parallelize([("a", 1), ("b", 1), ("a", 1)])
from operator import add
print(rdd.foldByKey(0, add).collect())

[('b', 1), ('a', 2)]


In [ ]:
RDD.groupByKeyでキーごとに値を連結
RDD.groupByKey(numPartitions=None, partitionFunc=<function portable_hash>)

In [ ]:
- pyspark.resultiterable.ResultIterableが返ってきてこれにlistやlenを引数として与えないとみれない

In [9]:
rdd = sc.parallelize([("a", 1), ("b", 1), ("a", 1)])
rdd.groupByKey().collect()

[('b', <pyspark.resultiterable.ResultIterable at 0x1912b155910>),
 ('a', <pyspark.resultiterable.ResultIterable at 0x1912b31ba30>)]

In [13]:
print(sorted(rdd.groupByKey().mapValues(len).collect()))
print(sorted(rdd.groupByKey().mapValues(list).collect()))

[('a', 2), ('b', 1)]
[('a', [1, 1]), ('b', [1])]


In [ ]:
RDD.sortByKey（ascending = True、numPartitions = None、keyfunc = <functionRDD。<lambda >> ）

In [1]:
tmp = [('a', 1), ('b', 2), ('1', 3), ('d', 4), ('2', 5)]
print(sc.parallelize(tmp).sortByKey().first())
print(sc.parallelize(tmp).sortByKey(True,numPartitions=1).collect())
print(sc.parallelize(tmp).sortByKey(True,numPartitions=2).collect())

tmp2 = [('Mary', 1), ('had', 2), ('a', 3), ('little', 4), ('lamb', 5)]
tmp2.extend([('whose', 6), ('fleece', 7), ('was', 8), ('white', 9)])
print(sc.parallelize(tmp2).sortByKey(True,numPartitions=3, keyfunc=lambda k: k.lower()).collect())


('1', 3)
[('1', 3), ('2', 5), ('a', 1), ('b', 2), ('d', 4)]
[('1', 3), ('2', 5), ('a', 1), ('b', 2), ('d', 4)]
[('a', 3), ('fleece', 7), ('had', 2), ('lamb', 5), ('little', 4), ('Mary', 1), ('was', 8), ('white', 9), ('whose', 6)]


In [14]:
pyspark.RDD.join
RDD.join(other, numPartitions=None)[source]

In [16]:
x = sc.parallelize([("a", 1), ("b", 4)])
y = sc.parallelize([("a", 2), ("a", 3)])
sorted(x.join(y).collect())

[('a', (1, 2)), ('a', (1, 3))]

In [13]:
RDD.persist（storageLevel = StorageLevel（False、True、False、False、1））[ソース]
このRDDのストレージレベルを設定して、最初に計算された後も操作間でその値を保持します。これは、RDDにストレージレベルがまだ設定されていない場合にのみ、新しいストレージレベルを割り当てるために使用できます。ストレージレベルが指定されていない場合、デフォルトで（MEMORY_ONLY）になります。

In [32]:
import pyspark
pyspark.StorageLevel.MEMORY_ONLY
pyspark.StorageLevel.DISK_ONLY


StorageLevel(True, False, False, False, 1)

In [ ]:
RDD.persist（storageLevel = StorageLevel（False、True、False、False、1））

In [38]:
import pyspark
rdd = sc.parallelize(["b", "a", "c"])
# rdd.persist(storageLevel=pyspark.StorageLevel.DISK_ONLY)
rdd.persist(storageLevel=pyspark.StorageLevel(True, False, False, False, 1))

ParallelCollectionRDD[76] at readRDDFromFile at PythonRDD.scala:274

In [52]:
rdd.unpersist()

ParallelCollectionRDD[77] at readRDDFromFile at PythonRDD.scala:274

In [70]:
a = sc.accumulator(1)
a.value

a.value = 2
a.value

a += 5
a.value

sc.accumulator(1.0).value


In [74]:

# sc.accumulator(3J).value
a = sc.accumulator(10)
a.value

rdd = sc.parallelize([1,2,3])
def f(x):
    global a
    a += x
    
rdd.foreach(f)
a.value

16

In [75]:
rdd.collect()

[1, 2, 3]

In [ ]:










b = sc.accumulator(0)
def g(x):
    b.add(x)
rdd.foreach(g)
b.value